In [1]:
import pandas as pd
import PIL
from tqdm import tqdm
from preprocess.pipelines import make_pipeline

/home/jupyter/MedClip/src/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def process_mimic(row): 
    """Process a row in the cxr-record-list.csv dataframe to 
    include caption and complete path to jpg image. 
    
    
    
    
    """
    try:
        text_path_prefix='../data/raw/physionet.org/files/mimic-cxr/2.0.0/'
        image_path_prefix= '../data/raw/physionet.org/files/mimic-cxr-jpg/2.0.0/'
        text_path_suffix=texts_df.loc[texts_df['study_id']==row['study_id']]['path'].iloc[0]

        # Obtain the report ---------------------------
        text_path=text_path_prefix+text_path_suffix
        # Read report into string
        text_file = open(text_path, "r")
        #read whole file to a string
        report = text_file.read()
        #close file
        text_file.close()
        row['Full_Caption']='<start> '+ report+' <end>'

        # get image path------------------------------
        row['Path']=image_path_prefix+row['path'][0:-3]+'jpg'
 
        return row[['dicom_id','Path','Full_Caption']]
    except:
        row['Full_Caption']='error'
        row['Path']='error'
        return row[['dicom_id','Path','Full_Caption']]

images_df_path="../data/raw/physionet.org/files/mimic-cxr/2.0.0/cxr-record-list.csv"
texts_df_path="../data/raw/physionet.org/files/mimic-cxr/2.0.0/cxr-study-list.csv"
labels_df_path="../data/raw/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-chexpert.csv"


images_df=pd.read_csv(images_df_path)
texts_df=pd.read_csv(texts_df_path)
labels_df=pd.read_csv(labels_df_path)

tqdm.pandas()
df=images_df.progress_apply(process_mimic,axis=1)
df.to_csv('../data/intermediate/mimic.csv')




 35%|███▍      | 131916/377110 [04:36<08:23, 487.00it/s]

In [3]:
pipeline=make_pipeline(inter_dataset_path='../data/intermediate/mimic.csv',
                  image_size=(299, 299),downscale=True,
                  shuffle=True,
                  seed=1,fractions=[0.70,0.15,0.15])

making input pipeline
